# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
       
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

#print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- 1.artist 
- 2.firstName of user
- 3.gender of user
- 4.item number in session
- 5.last name of user
- 6.length of the song
- 7.level (paid or free song)
- 8.location of the user
- 9.sessionId
- 10.song title
- 11.userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [5]:
## Visualising the data in a dataframe
df = pd.read_csv('event_datafile_new.csv')
df.sample(5)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
3427,Helloween,Layla,F,34,Griffin,345.36444,paid,"Lake Havasu City-Kingman, AZ",984,Back Against The Wall,24
6578,Antonio Vega,Tegan,F,27,Levine,135.13098,paid,"Portland-South Portland, ME",910,Sin SoluciÃÂ³n,80
5538,BjÃÂ¶rk,Mohammad,M,24,Rodriguez,185.46893,paid,"Sacramento--Roseville--Arden-Arcade, CA",744,Cover Me,88
6598,Aqua,Aiden,M,4,Hess,218.48771,free,"La Crosse-Onalaska, WI-MN",906,Cartoon Heroes,86
3324,Coldplay,Kate,F,36,Harrell,368.09098,paid,"Lansing-East Lansing, MI",944,Fix You (Live),97


## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## The daa exploration revealed that each sessionId has 1 itemInSession making the pair to be a unique identifier.
## Then the rest of the required data is retrieved with the primary key.

query = "CREATE TABLE IF NOT EXISTS song_in_session"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, \
                    PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_in_session (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
query = "select artist, song, length from song_in_session where sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [12]:
## checking the results with pandas
df[(df.sessionId == 338) & (df.itemInSession == 4)][['artist', 'song', 'length']]

,artist,song,length
4895,Faithless,Music Matters (Mark Knight Dub),495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## The userId is the partition key and the sessionId and itemInSession are the clustering columns that sort the data for \
## each user.

query = "CREATE TABLE IF NOT EXISTS song_listen_by_user"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, \
                    PRIMARY KEY (userId, sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listen_by_user (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [15]:
query = "select artist, song, firstName, lastName from song_listen_by_user where userId = 10 and sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [16]:
## checking the results with pandas
df[(df.userId == 10) & (df.sessionId == 182)]\
        [['itemInSession', 'artist', 'song', 'firstName', 'lastName']]\
        .sort_values(by='itemInSession')

,itemInSession,artist,song,firstName,lastName
1475,0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1476,1,Three Drives,Greece 2000,Sylvie,Cruz
1477,2,Sebastien Tellier,Kilometer,Sylvie,Cruz
1478,3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [17]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## The song and the userId makes it a unique identifier to handle the query even if the song is \
## listened by the same user in a different sessionId will not effect the result of the query. 

query = "CREATE TABLE IF NOT EXISTS users_listened_to_song"
query = query + "(song text, userId int, firstName text, lastName text, \
                    PRIMARY KEY (song, userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)                     

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_listened_to_song (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, ( line[9], int(line[10]), line[1], line[4]))

In [19]:
query = "select firstName, lastName from users_listened_to_song where song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [20]:
## checking the results with pandas
df[df.song == 'All Hands Against His Own']\
        [['firstName', 'lastName', 'userId']]\
        .sort_values(by='userId')

,firstName,lastName,userId
4412,Jacqueline,Lynch,29
602,Tegan,Levine,80
305,Sara,Johnson,95


### Drop the tables before closing out the sessions

In [21]:
## TO-DO: Drop the table before closing out the sessions

query = "DROP table song_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP table song_listen_by_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP table users_listened_to_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()